**Unique Category File**

In [1]:
# For main_product file

import pandas as pd

df = pd.read_excel('main_product.xlsx',dtype=str)

# Split the values in each row by comma and find unique values row-wise
df["unique_row_values"] = df["category_name"].apply(lambda x: list(set(x.split(','))))

# Combine all rows into a single list to find unique values across the entire column
unique_values = set()
df["unique_row_values"].apply(lambda x: unique_values.update(x))

# Convert the set of unique values to a list and create a new DataFrame
unique_category_df = pd.DataFrame(sorted(list(unique_values)), columns=["unique_category"])

# Display the new DataFrame
unique_category_df


,unique_category
0,BUY 3 GET 10%OFF
1,CURTIS15PICKS 2
2,MADEIRA
3,SPIRITS EDU
4,WINE
...,...
130,WINES FOR GRAND TESTING
131,WOTW 12 QTY DISCOUNT
132,XXL MOSCATO 2 FOR 18
133,base_price_zero


In [ ]:
import pandas as pd

df = pd.read_excel('variants.xlsx',dtype=str)

# Split the values in each row by comma and find unique values row-wise
df["unique_row_values"] = df["v_category"].apply(lambda x: list(set(x.split(','))))

# Combine all rows into a single list to find unique values across the entire column
unique_values = set()
df["unique_row_values"].apply(lambda x: unique_values.update(x))

# Convert the set of unique values to a list and create a new DataFrame
unique_category_df2 = pd.DataFrame(sorted(list(unique_values)), columns=["unique_category"])

# Display the new DataFrame
unique_category_df2


In [ ]:
print('product_unique_category_count',unique_category_df.shape)
print('variants_unique_category_count',unique_category_df2.shape)


In [ ]:
merged_df = pd.concat([unique_category_df, unique_category_df2]).drop_duplicates().reset_index(drop=True)
merged_df.shape

In [ ]:
unique_category_df2.to_excel('category_name(unique).xlsx',index=False)

**Category Count**

In [ ]:
import pandas as pd
from collections import Counter

def process_category_file(filename, column_name):
    """
    Reads an Excel file, extracts unique category values, and counts occurrences.

    Parameters:
    filename (str): Path to the Excel file.
    column_name (str): Name of the category column in the file.

    Returns:
    pd.DataFrame: DataFrame containing unique categories and their counts.
    """
    # Load the Excel file
    df = pd.read_excel(filename, dtype=str)

    # Ensure the column exists in the file
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in {filename}")

    # Split values row-wise and get unique categories per row
    df["unique_row_values"] = df[column_name].apply(lambda x: list(set(x.split(','))) if pd.notna(x) else [])

    # Flatten the list of all categories
    all_categories = [item.strip() for sublist in df["unique_row_values"] for item in sublist]

    # Count occurrences of each category
    category_counts = Counter(all_categories)

    # Convert to DataFrame
    unique_category_df = pd.DataFrame(category_counts.items(), columns=["unique_category", "count"])

    # Sort by category name
    unique_category_df = unique_category_df.sort_values(by="unique_category").reset_index(drop=True)

    return unique_category_df

# Example usage
main_product_df = process_category_file("main_product.xlsx", "category_name")
variants_df = process_category_file("variants.xlsx", "v_category")

# Display results
main_product_df

In [ ]:
variants_df

In [2]:
main_product_df.to_excel('main_product_category_name(count).xlsx',index=False)
variants_df.to_excel('variants_category_name(count).xlsx',index=False)


**Update Category name by adding [' - 01', ' - 02'] for count > 4500**

In [1]:
import pandas as pd

df1 = pd.read_excel('main_product.xlsx', dtype=str)
df2 = pd.read_excel('variants.xlsx', dtype=str)

df1['file'] = 'main_product'
df2['file'] = 'variants'

df1['index_id'] = [f'mp_{i+1}' for i in range(len(df1))]
df2['index_id'] = [f'v_{i+1}' for i in range(len(df2))]
df2['category_name'] = df2['v_category']

df = pd.concat([df1[['index_id','category_name','file']],df2[['index_id','category_name','file']]], ignore_index=True)

# Dictionary to track category occurrences
category_count = {}

def process_category(row):
    categories = row.split(",")  # Split values
    updated_categories = []

    for cat in categories:
        if cat in category_count:
            category_count[cat] += 1
        else:
            category_count[cat] = 1

        # Append suffix if count > 4500
        if category_count[cat] > 4500:
            suffix = f"- {(category_count[cat] - 1) // 4500:02d}"
            updated_categories.append(f"{cat} {suffix}")
        else:
            updated_categories.append(cat)

    return ",".join(updated_categories)

df["new_category"] = df["category_name"].apply(process_category)

df


,index_id,category_name,file,new_category
0,mp_1,"BEER,CRAFT BEER,ALL MENU",main_product,"BEER,CRAFT BEER,ALL MENU"
1,mp_2,"CRAFT BEER,ALL MENU",main_product,"CRAFT BEER,ALL MENU"
2,mp_3,"BEER,CRAFT BEER,ALL MENU",main_product,"BEER,CRAFT BEER,ALL MENU"
3,mp_4,"BEER,IMPORTED BEER",main_product,"BEER,IMPORTED BEER"
4,mp_5,"BEER,CRAFT BEER",main_product,"BEER,CRAFT BEER"
...,...,...,...,...
33329,v_17323,"BEER,CRAFT BEER,ALL MENU",variants,"BEER - 02,CRAFT BEER - 02,ALL MENU - 06"
33330,v_17324,"MISCELLANEOUS,MISCELLANEOUS,ALL MENU",variants,"MISCELLANEOUS,MISCELLANEOUS,ALL MENU - 06"
33331,v_17325,"MISCELLANEOUS,MISCELLANEOUS,ALL MENU",variants,"MISCELLANEOUS,MISCELLANEOUS,ALL MENU - 06"
33332,v_17326,"CORDIALS/LIQUEURS,ALL MENU",variants,"CORDIALS/LIQUEURS,ALL MENU - 06"


In [3]:
df1.to_excel('new_file/main_product_new.xlsx',index=False)
df2.to_excel('new_file/variant_new.xlsx',index=False)
df.to_excel('new_file/category_name_new.xlsx',index=False)

NOTE:-

After STORING above **Three DataFrames** [df1,df2, df] in files
We have to map new_category column from category_name_new file by index_id.

For **main_product**:
1. Using the index_id column in the main_product_new file,
   map the new_category value from the category_name_new file.
2. Remove any unnecessary columns from the main_product_new file.

For **variants**:
1. Using the index_id column in the variant_new file,
   map the new_category value from the category_name_new file.
2. Remove any unnecessary columns from the variant_new file.